## 데이터 불러오기

In [44]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
import random
import optuna
import glob
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold,train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
train = pd.read_csv('../data/train_ver2.csv', index_col=0)
test = pd.read_csv('../data/test_ver2.csv', index_col=0)

In [3]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [4]:
X = train.drop(["credit"], axis=1)
y = train["credit"]
X_test = test.copy()

## XGBoost

### 하이퍼파라미터 튜닝 - optuna

In [10]:
def objective_xgb(trial: Trial) -> float:
    params_xgb = {
        "random_state": 42,
        "learning_rate": trial.suggest_discrete_uniform('learning_rate', 0.01, 0.1, 0.01),
        "n_estimators": trial.suggest_int('n_estimators', 0, 1000),
        "objective": "multiclass",
        "metric": "multi_logloss",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "colsample_bytree": trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1),
        "subsample": trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
    }
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

    xgb = XGBClassifier(**params_xgb)
    xgb.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=30,
        verbose=100,
    )

    xgb_pred = xgb.predict_proba(X_valid)
    log_score = log_loss(y_valid, xgb_pred)
    
    return log_score

In [6]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'xgb_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)
study.optimize(objective_xgb, n_trials=10)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2021-12-02 14:59:06,504] A new study created in memory with name: xgb_parameter_opt


[14:59:06] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:59:06] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.08228	validation_1-mlogloss:1.08250
[100]	validation_0-mlogloss:0.78763	validation_1-mlogloss:0.80636
[200]	validation_0-mlogloss:0.75890	validation_1-mlogloss:0.79082
[300]	validation_0-mlogloss:0.73542	validation_1-mlogloss:0.77898
[400]	validation_0-mlogloss:0.71585	validation_1-mlogloss:0.77049
[500]	validation_0-mlogloss:0.69869	validation_1-mlogloss:0.76360
[600]	validation_0-mlogloss

[I 2021-12-02 14:59:22,879] Trial 0 finished with value: 0.7413744503749066 and parameters: {'learning_rate': 0.04, 'n_estimators': 951, 'reg_alpha': 2.196249831492404e-05, 'reg_lambda': 0.05387926759114846, 'max_depth': 4, 'colsample_bytree': 0.5, 'subsample': 0.5}. Best is trial 0 with value: 0.7413744503749066.


[14:59:22] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:59:23] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.03270	validation_1-mlogloss:1.04975
[61]	validation_0-mlogloss:0.23176	validation_1-mlogloss:0.74999


[I 2021-12-02 14:59:32,129] Trial 1 finished with value: 0.723628764196546 and parameters: {'learning_rate': 0.09, 'n_estimators': 601, 'reg_alpha': 2.1245096608103405e-05, 'reg_lambda': 0.0018526142807772773, 'max_depth': 20, 'colsample_bytree': 0.9, 'subsample': 0.6}. Best is trial 1 with value: 0.723628764196546.


...

[15:01:14] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:01:14] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.09363	validation_1-mlogloss:1.09398
[100]	validation_0-mlogloss:0.84427	validation_1-mlogloss:0.87170
[195]	validation_0-mlogloss:0.76274	validation_1-mlogloss:0.81155


[I 2021-12-02 15:01:22,621] Trial 8 finished with value: 0.8115462687037613 and parameters: {'learning_rate': 0.01, 'n_estimators': 196, 'reg_alpha': 1.3663663944270366e-06, 'reg_lambda': 0.029279736515390484, 'max_depth': 8, 'colsample_bytree': 0.6, 'subsample': 0.9}. Best is trial 3 with value: 0.691593056951268.


[15:01:22] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:01:22] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.07038	validation_1-mlogloss:1.07846
[100]	validation_0-mlogloss:0.34923	validation_1-mlogloss:0.72486
[122]	validation_0-mlogloss:0.30652	validation_1-mlogloss:0.72991


[I 2021-12-02 15:01:34,652] Trial 9 finished with value: 0.7244031605557343 and parameters: {'learning_rate': 0.04, 'n_estimators': 281, 'reg_alpha': 1.6285455533915874e-05, 'reg_lambda': 0.01268318883848639, 'max_depth': 17, 'colsample_bytree': 0.5, 'subsample': 0.9}. Best is trial 3 with value: 0.691593056951268.


Best Score: 0.691593056951268
Best trial {'learning_rate': 0.02, 'n_estimators': 292, 'reg_alpha': 1.0997191680377813e-05, 'reg_lambda': 0.04104630401883339, 'max_depth': 16, 'colsample_bytree': 0.5, 'subsample': 0.7}


### 10-Fold + XGBoost

In [12]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx,valid_idx))

In [14]:
random.seed(42)
xgb_models={}

for fold in range(10):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train = train.drop(['credit'],axis=1).iloc[train_idx].values 
    X_valid = train.drop(['credit'],axis=1).iloc[valid_idx].values
    y_train = train['credit'][train_idx].values
    y_valid = train['credit'][valid_idx].values

    xgb = XGBClassifier(**study.best_params)
    xgb.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=30,
        verbose=100,
    )
    xgb_models[fold] = xgb
    print(f'================================================================================\n\n')

====================================1============================================
[15:27:53] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.08683	validation_1-mlogloss:1.08936
[100]	validation_0-mlogloss:0.55497	validation_1-mlogloss:0.73509
[200]	validation_0-mlogloss:0.40527	validation_1-mlogloss:0.69271
[264]	validation_0-mlogloss:0.34324	validation_1-mlogloss:0.69070


====================================2============================================
[15:28:17] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.08577	validation_1-mlogloss:1.08914
[1

In [22]:
submit = pd.read_csv('../data/sample_submission.csv')

In [23]:
submit.iloc[:,1:]=0
for fold in range(10):
    submit.iloc[:,1:] += xgb_models[fold].predict_proba(test)/10

In [24]:
submit.describe()

,index,0,1,2
count,9998.00000,9998.000000,9998.000000,9998.000000
mean,31455.50000,0.113125,0.207456,0.679419
std,2886.31833,0.068474,0.160625,0.183808
min,26457.00000,0.019798,0.023012,0.024428
25%,28956.25000,0.072517,0.119283,0.636246
50%,31455.50000,0.094281,0.158578,0.729430
75%,33954.75000,0.129920,0.222848,0.790420
max,36454.00000,0.733110,0.940980,0.957190


In [25]:
submit.to_csv('../data/submit_xgb.csv', index=False) # test 데이터 전처리 과정에서 row 2개가 빠져 평균값으로 채워주었다 (대회 score 제출 규정)

## RandomForest

### 하이퍼파라미터 튜닝 - optuna

In [47]:
def objective_rf(trial: Trial) -> float:
    params_rf = {
        "random_state": 42,
        "n_estimators": trial.suggest_int('n_estimators', 50, 1000),
        "max_depth": trial.suggest_int('max_depth', 4, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 1, 150),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
    }
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

    rf = RandomForestClassifier(**params_rf)
    rf.fit(X_train, y_train)
    
    rf_pred = rf.predict_proba(X_valid)
    log_score = log_loss(y_valid, rf_pred)
    
    return log_score

In [48]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'rf_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)
study.optimize(objective_rf, n_trials=10)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2021-12-02 16:37:46,529] A new study created in memory with name: rf_parameter_opt
[I 2021-12-02 16:37:55,193] Trial 0 finished with value: 0.7908607815252745 and parameters: {'n_estimators': 406, 'max_depth': 48, 'min_samples_split': 110, 'min_samples_leaf': 36}. Best is trial 0 with value: 0.7908607815252745.
[I 2021-12-02 16:37:58,616] Trial 1 finished with value: 0.7853724802341039 and parameters: {'n_estimators': 198, 'max_depth': 11, 'min_samples_split': 9, 'min_samples_leaf': 52}. Best is trial 1 with value: 0.7853724802341039.
[I 2021-12-02 16:38:10,676] Trial 2 finished with value: 0.7989344695596716 and parameters: {'n_estimators': 621, 'max_depth': 37, 'min_samples_split': 4, 'min_samples_leaf': 59}. Best is trial 1 with value: 0.7853724802341039.
[I 2021-12-02 16:38:28,373] Trial 3 finished with value: 0.7874047977497097 and parameters: {'n_estimators': 841, 'max_depth': 13, 'min_samples_split': 28, 'min_samples_leaf': 12}. Best is trial 1 with value: 0.7853724802341039.

Best Score: 0.7784112114929467
Best trial {'n_estimators': 483, 'max_depth': 40, 'min_samples_split': 30, 'min_samples_leaf': 31}


### 10-Fold + XGBoost

In [49]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx,valid_idx))

In [50]:
random.seed(42)
rf_models={}

for fold in range(10):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train = train.drop(['credit'],axis=1).iloc[train_idx].values 
    X_valid = train.drop(['credit'],axis=1).iloc[valid_idx].values
    y_train = train['credit'][train_idx].values
    y_valid = train['credit'][valid_idx].values

    rf = RandomForestClassifier(**study.best_params)
    rf.fit(X_train, y_train)
    rf_models[fold] = rf
    print(f'================================================================================\n\n')

====================================1============================================


====================================2============================================


====================================3============================================


====================================4============================================


====================================5============================================


====================================6============================================


====================================7============================================


====================================8============================================


====================================9============================================


====================================10============================================


